In [ ]:
#export
import torch.nn as nn
from Functions import *

In [ ]:
# Load all the data from the CSV file 
PT_DATA_PATH = "../../../Dataset/Botnet_Detection/PT_838_Security Camera"
PT2_DATA_PATH = "../../../Dataset/Botnet_Detection/PT737E_Security Camera"
XC_DATA_PATH = "../../../Dataset/Botnet_Detection/XCS7_1002_WHT_Security_Camera"
XC2_DATA_PATH = "../../../Dataset/Botnet_Detection/XCS7_1003_WHT_Security_Camera"
df_pt_1 = pd.read_csv(PT_DATA_PATH+"/benign_traffic.csv")
df_pt_2 = pd.read_csv(PT2_DATA_PATH+"/benign_traffic.csv")
df_xc_1 =  pd.read_csv(XC_DATA_PATH+"/benign_traffic.csv")
df_xc_2 =  pd.read_csv(XC2_DATA_PATH+"/benign_traffic.csv")

In [ ]:
df_pt1 = df_pt_1.assign(label = 'pt1')
df_pt2 = df_pt_2.assign(label = 'pt2')
df_xc1 = df_xc_1.assign(label = 'xc1')
df_xc2 = df_xc_2.assign(label = 'xc2')

## Prepare for OMP

In [ ]:
df_all = df_pt1
#df_all = df_all.append(df_pt2, ignore_index = True)

In [ ]:
df_all = shuffler(df_all)

In [ ]:
#Sample some instances from the dataset 
test_set = df_pt1.sample(frac=0.001, random_state=1)
test_set = test_set.append(df_xc1.sample(frac=0.001, random_state=1))
#test_set = shuffler(test_set)
print(test_set.shape)

In [ ]:
train_x = pd.DataFrame(df_all.iloc[:, 0:115])
train_y = pd.DataFrame(df_all.iloc[:, 115])
test_x = pd.DataFrame(test_set.iloc[:, 0:115])
test_y = pd.DataFrame(test_set.iloc[:, 115])

In [ ]:
train_x = normalize(train_x)
test_x = normalize(test_x)

In [ ]:
train_y = label_encoder(train_y)
test_y = label_encoder(test_y)

In [ ]:
test_y

In [ ]:
train_x = torch.FloatTensor(train_x.values.astype(np.float32))
test_x = torch.tensor(test_x.values.astype(np.float32))
train_y = torch.tensor(train_y.astype(np.float32))
test_y = torch.tensor(test_y.astype(np.float32))

## Prepare for the NN

In [ ]:
df_all = df_pt1
df_all = df_all.append(df_pt2, ignore_index = True)

In [ ]:
df_all = shuffler(df_all)

In [ ]:
#Divide dataframe into x and y
df_x = pd.DataFrame(df_all.iloc[:, 0:115])
df_y = pd.DataFrame(df_all.iloc[:, 115])
df_pt2_x = pd.DataFrame(df_pt2.iloc[:, 0:115])
df_pt2_y = pd.DataFrame(df_pt2.iloc[:, 115])
df_xc_x = pd.DataFrame(df_xc1.iloc[:, 0:115])
df_xc_y = pd.DataFrame(df_xc1.iloc[:, 115])
df_xc2_x = pd.DataFrame(df_xc2.iloc[:, 0:115])
df_xc2_y = pd.DataFrame(df_xc2.iloc[:, 115])

In [ ]:
#Normalize the x dataframe
df_x = normalize(df_x)
df_pt2 = normalize(df_pt2_x)
df_xc = normalize(df_xc_x)
df_xc2 = normalize(df_xc2_x)

In [ ]:
#One-Hot encoding labels and transform into array
df_y = label_encoder(df_y)

In [ ]:
# Divide dataset into training set and testing set
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(df_x, df_y, test_size=0.40)

## Neural Network

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_dim, h_dim, out_dim):
        super(Net, self).__init__()
        self.linear1 = torch.nn.Linear(in_dim, h_dim)
        self.bn1 = nn.BatchNorm1d(h_dim)
        self.linear2 = torch.nn.Linear(h_dim, out_dim)
        
    def forward(self, x):
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

In [ ]:
train_x = torch.FloatTensor(train_x.values.astype(np.float32))
test_x = torch.tensor(test_x.values.astype(np.float32))
train_y = torch.tensor(train_y.astype(np.float32))
test_y = torch.tensor(test_y.astype(np.float32))
xc_x = torch.tensor(df_xc.values.astype(np.float32))
pt2_x = torch.tensor(df_pt2.values.astype(np.float32))

In [ ]:
train_x.shape

In [ ]:
test_x.shape

In [ ]:
test_y.shape

In [ ]:
epochs = 600
input_dim = 115
output_dim = 2 #Number of clasees
h_dim = 100
lr_rate = 1e-6

In [ ]:
model = Net(input_dim, h_dim, output_dim)
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [ ]:
n_test, y_bm = test_y.shape

In [ ]:
def training(epochs, model, train_data, labels, vali_data, target):
    for e in range(int(epochs)):
        y_pred = model(train_data)
        # Compute and print loss
        loss = criterion(y_pred, labels)
        if e % 100 == 99:
            loss_f = float(loss)
            print(e, "Loss:", loss_f)
            total = n_test
            correct = 0.0
            outputs = model(vali_data)
            _b, pred = torch.max(outputs, 1)
            vb, label = torch.max(target, 1)
            correct+= float((pred == label).sum())
            accuracy = float(100*(correct/total))
            print('Accuracy: {:.4f}'.format(accuracy))

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
training(epochs, model, train_x, train_y, test_x, test_y) ## Train the initial model on Server

## OMP

In [ ]:
def maxind(V, r):
    res=[]
    for row in V:
        l2 = np.linalg.norm(row)
        row = row.unsqueeze(1)
        row = torch.transpose(row,0,1)       
        row = row.squeeze(0)
        r = r.squeeze(0)
        inner = float(torch.dot(row, r))
        s = float(inner/l2)
        res.append(s)
        res_tensor = torch.FloatTensor(res)
    
    _, i = torch.max(res_tensor, 0)
    return i

In [ ]:
## Note: The vector(row) picked from the matrix cannot be re-sized directly.
## y is expected as 2D matrix
def OMP(y, V, sl, ep):
    c_encode = torch.zeros([1, len(V)], dtype=torch.float32)
    s_index = []
    r = y
    k = 1
    while (k <= sl) and (np.linalg.norm(r) >= ep):
        i = int(maxind(V, r))
        s_index.append(i)
        temp = []
        for sk in s_index:
            temp_v = V[sk]
            if len(temp)==0:
                temp=temp_v
                temp = temp.unsqueeze(0)
            else:
                temp_v = V[sk].unsqueeze(0)
                temp = torch.cat((temp, temp_v),0)
               
        
        s_matrix = temp
        s_matrix = s_matrix.type(torch.FloatTensor)
        s_i = torch.pinverse(s_matrix)
        c = torch.mm(y, s_i)
        r = y - torch.mm(c, s_matrix)
        k = k+1
    
    print(s_index)
    v=0
    for sk in s_index:
        c_encode[0][sk] = c[0][v]
        v = v+1
    return c_encode

In [ ]:
OMP(b, a, 20, 0.1)

In [ ]:
y = test_x[5][:].unsqueeze(0)

In [ ]:
c = OMP(y, train_x[:300][:], 10, 0.001)

In [ ]:
t = y - torch.mm(c, train_x[:300][:])
print(np.linalg.norm(t))

In [ ]:
def projecting(train, test, size, sp, ep, threshold):
    re = torch.zeros([len(test), 2], dtype=torch.float32)
    for ids, vector in enumerate(test):
        v = vector.unsqueeze(0)
        c = OMP(v, train[:size][:], sp, ep)
        t = v - torch.mm(c, train[:size][:])
        distance = np.linalg.norm(t)
        print("id:", ids, "dist:",distance)
        if distance < threshold:
            re[ids][0] = 1.0
        else:
            re[ids][1] = 1.0
    print(re)

In [ ]:
total = len(test_x)
correct = 0.0
_, pred = torch.max(re, 1)
v, label = torch.max(test_y, 1)
correct+= float((pred == label).sum())
accuracy = float(100*(correct/total))
print('Accuracy: {:.4f}'.format(accuracy))